In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [2]:
path = './data/'

## 컴피티션 설명
마을의 익숙치 않은 지역에서 배가 고프며 자신의 취향에 따라 적절한 순간에 레스토랑 추천을 받았다고 상상해보십시오. 추천서는 모퉁이에있는 현지 장소에 대한 신용 카드 제공 업체의 할인을 제공합니다!

현재 브라질의 최대 결제 브랜드 중 하나인 Elo 는 카드 회원에 대한 프로모션 또는 할인을 제공하기 위해 상인과의 파트너십을 구축했습니다. 하지만이 프로모션은 소비자 또는 판매자 모두에게 효과가 있습니까? 고객은 그들의 경험을 즐긴다? 상인들은 반복되는 사업을 보나요? 개인화가 중요합니다.

Elo는 음식부터 쇼핑에 이르기까지 고객의 라이프 사이클에서 가장 중요한 측면과 선호 사항을 이해하기 위해 기계 학습 모델을 구축했습니다. 그러나 지금까지는 개인이나 프로필을 위해 특별히 맞춤 설정된 것은 없습니다. 이것은 당신이 들어오는 곳입니다.

이 경쟁에서, Kagglers는 고객 충성도의 신호를 밝힘으로써 개인에게 가장 관련성 높은 기회를 식별하고 제공하는 알고리즘을 개발할 것입니다. 귀하의 의견은 고객의 삶을 개선하고 Elo가 원치 않는 캠페인을 줄이고 고객을위한 올바른 경험을 창출하도록 도와줍니다.

## 데이터에 대한 설명

historical_transactions.csv 및 new_merchant_transactions.csv 파일은 각 카드의 거래에 대한 정보가 포함되어 있습니다.

* historical_transactions.csv에는 각 카드 id별 3개월 거래 데이터 포함

* new_merchant_transactions.csv에는 historical 데이터에 한번도 반영되지 않은 상인 아이디에 의해 생성된 각 카드 id별 2개월 거래 데이터

* merchants.csv 에는 merchant_id 표시된 상인에 대한 집계 정보

## 데이터 로드

In [3]:
train = pd.read_csv(path + 'train.csv')
history = pd.read_csv(path + 'historical_transactions.csv')
merchant_transaction = pd.read_csv(path + 'new_merchant_transactions.csv')
merchant = pd.read_csv(path + 'merchants.csv')

### train 컬럼

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null object
card_id               201917 non-null object
feature_1             201917 non-null int64
feature_2             201917 non-null int64
feature_3             201917 non-null int64
target                201917 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB


* first_active_month: 거래 최초 발생 시간, 년 월 표시
* card_id: 고유한 카드 식별 정보
* feature_1: ?
* feature_2: ?
* feature_3: ?
* target: 평가기간 2개월 후에 계산되는 충성도 점수

In [5]:
train.shape

(201917, 6)

In [6]:
train.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056


### histroy 컬럼

In [7]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
authorized_flag         object
card_id                 object
city_id                 int64
category_1              object
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           object
category_2              float64
state_id                int64
subsector_id            int64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.0+ GB


* authorized_flag: 거래요청 승인 내역
* card_id: 고유 카드 식별자
* city_id: 고유 도시 식별자
* category_1: ?
* installments: 할부 개월 수
* category_3: ?
* merchant_category_id: 상인 카테고리 식별자
* merchant_id: 상인 식별자
* month_lag: 기준일까지 지체시간 잘 모르겠음 ...
* purchase_amont: 정규화된 거래 금액
* purchase_date: 거래 날짜
* category_2: ?
* state_id: state 식별자라는데 주 정보인지? 브라질에 주가 있나요? 한번 확인해볼게용
* subsector_id: 상인 카테고리 그룹 식별자

In [8]:
history.shape

(29112361, 14)

In [9]:
history.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37


### new merchant 컬럼
히스토리랑 똑같음. 다만, 신규 오픈한 가게인지 아닌지 여부에 따라 구분

### merchant 컬럼

In [10]:
merchant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
merchant_id                    334696 non-null object
merchant_group_id              334696 non-null int64
merchant_category_id           334696 non-null int64
subsector_id                   334696 non-null int64
numerical_1                    334696 non-null float64
numerical_2                    334696 non-null float64
category_1                     334696 non-null object
most_recent_sales_range        334696 non-null object
most_recent_purchases_range    334696 non-null object
avg_sales_lag3                 334683 non-null float64
avg_purchases_lag3             334696 non-null float64
active_months_lag3             334696 non-null int64
avg_sales_lag6                 334683 non-null float64
avg_purchases_lag6             334696 non-null float64
active_months_lag6             334696 non-null int64
avg_sales_lag12                334683 non-null float64
avg_purchases_lag12    

* merchant_id: 상인 아이디
* merchnat_group_id: 상인 그룹 아이디
* merchant_category_id: 상인 카테고리 식별자
* subsector_id: 상인 카테고리 그룹
* numerical_1: ?
* numerical_2: ?
* category_1: ?
* most_recent_sales_range: 마지막 거래 월의 수익 단위 (A ~ E)
* most_recent_purchases_range: 마지막 거래 월의 거래 수량 단위 (A ~ E)
* avg_sales_lag3: 최근 3개월동안 수익의 평균을 마지막 거래 수익으로 나눈 값
* avg_purchases_lag3: 최근 3개월동안 거래량의 평균을 마지막 거래량 수익으로 나눈 값
* active_months_lag3: 지난 세 달동안 거래가 빈번한 달의 수? 양?
* 6은 위 3개를 3개월을 6개월로 바꾸면 됩니다.
* 12는 위 3개를 3개월을 12개월로 바꾸면 됩니다.
* category_4: ?
* city_id: 도시 정보
* state_id: 주? 정보
* category_2: ?

In [11]:
merchant.shape

(334696, 22)

In [12]:
merchant.head(3)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
